In [21]:
import os
import random

import pathlib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import skimage
from skimage import io as skimg_io
from skimage.transform import resize

import sklearn
from sklearn import  linear_model, svm, metrics

In [3]:
%matplotlib inline

In [13]:
IMAGE_W = 64
IMAGE_H = 64

In [4]:
images_dir = '../input/train'
train_images_description_file_path = '../input/train.csv'

In [5]:
train_images_description_df = pd.read_csv(train_images_description_file_path)
print(train_images_description_df.head())
print(train_images_description_df.shape)

           Image         Id
0  0000e88ab.jpg  w_f48451c
1  0001f9222.jpg  w_c3d896a
2  00029d126.jpg  w_20df2c5
3  00050a15a.jpg  new_whale
4  0005c1ef8.jpg  new_whale
(25361, 2)


In [6]:
train_images_description_df_two_classes = train_images_description_df.copy()

In [8]:
train_images_description_df_two_classes.loc[train_images_description_df_two_classes['Id'] != 'new_whale', 'Id'] = 'not_new_whale'

In [9]:
train_images_description_df_two_classes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25361 entries, 0 to 25360
Data columns (total 2 columns):
Image    25361 non-null object
Id       25361 non-null object
dtypes: object(2)
memory usage: 396.3+ KB


In [10]:
train_images_description_df_two_classes['NumId'] = 0

In [11]:
train_images_description_df_two_classes.loc[train_images_description_df_two_classes['Id'] != 'new_whale', 'NumId'] = 1

In [12]:
train_images_description_df_two_classes.head()

,Image,Id,NumId
0,0000e88ab.jpg,not_new_whale,1
1,0001f9222.jpg,not_new_whale,1
2,00029d126.jpg,not_new_whale,1
3,00050a15a.jpg,new_whale,0
4,0005c1ef8.jpg,new_whale,0


In [19]:
for row in train_images_description_df_two_classes.head().iterrows():
    print(type(row[1]))
    print(len(row[1]))
    print(row[1])

<class 'pandas.core.series.Series'>
3
Image    0000e88ab.jpg
Id       not_new_whale
NumId                1
Name: 0, dtype: object
<class 'pandas.core.series.Series'>
3
Image    0001f9222.jpg
Id       not_new_whale
NumId                1
Name: 1, dtype: object
<class 'pandas.core.series.Series'>
3
Image    00029d126.jpg
Id       not_new_whale
NumId                1
Name: 2, dtype: object
<class 'pandas.core.series.Series'>
3
Image    00050a15a.jpg
Id           new_whale
NumId                0
Name: 3, dtype: object
<class 'pandas.core.series.Series'>
3
Image    0005c1ef8.jpg
Id           new_whale
NumId                0
Name: 4, dtype: object


In [22]:
images = [
    (row[1]['Image'], row[1]['NumId'], resize(
        skimg_io.imread(
            pathlib.Path(images_dir, row[1]['Image']).as_posix()),
            (IMAGE_W, IMAGE_H)
        )
    ) for row in train_images_description_df_two_classes.iterrows()]

FileNotFoundError: [Errno 2] No such file or directory: '0000e88ab.jpg'